<h1> Prepare Data </h1>
To prepare the data for our models we need to:
<ol>
<li> cut unnecessary rows from the csv's (the rows that will not be used as input for the model, as explained in the instructions) </li>
<li> create train and validation datasets </li>
</ol>
<h2> Adding Columns </h2>
For each patients (and row) we will add 3 columns to the data frame:
<ol>
<li> max_ICULOS - the total time a patient was in the ICU </li>
<li> time_bm - the difference between the current time and the total time a patient was in the ICU. defined as $time_bm = ICULOS-max ICULOS$
</li>
<li> Label column -  1 if the patient had sepsis after some time in the ICU and 0 otherwise
</li>
</ol>


In [1]:
import pandas as pd
import os
import tqdm
from random import sample
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
TRAIN_PATH = 'filtered_train_df_0705'
VAL_PATH = 'filtered_val_df_0705'
TEST_PATH = 'filtered_test_df_0705'
TRAIN_MEAN_PATH = 'filtered_train_mean.csv'

In [3]:
# def create_patients_df(patients, data_path):
#     tmp_df = pd.read_csv(os.path.join(data_path,patients[0]),delimiter ='|')
#     tmp_df['ID'] = patients[0].split('_')[-1].split('.')[0]
#     new_df = tmp_df[tmp_df['SepsisLabel']==0]
#     if max(tmp_df['SepsisLabel'])==1:
#         new_df=new_df.append(tmp_df[tmp_df['SepsisLabel']==1][:1])
#         new_df['Label'] = [1]*new_df.shape[0]
#     else:
#         new_df['Label'] = [0]*new_df.shape[0]
#     new_df['max_ICULOS'] = [new_df['ICULOS'].values[-1]]*new_df.shape[0]
#     new_df['time_bm'] =  new_df['ICULOS']-new_df['max_ICULOS']
#     for patient in tqdm.tqdm(patients[1:]):
#         patient_path = os.path.join(data_path,patient)
#         patient_number = patient.split('_')[-1].split('.')[0]
#         tmp_df = pd.read_csv(os.path.join(data_path,patient_path),delimiter ='|')
#         tmp_df['ID'] = patient_number
#         tmp_new_df = tmp_df[tmp_df['SepsisLabel']==0]
#         if max(tmp_df['SepsisLabel'])==1:
#             tmp_new_df=tmp_new_df.append(tmp_df[tmp_df['SepsisLabel']==1][:1])
#             tmp_new_df['Label'] = [1]*tmp_new_df.shape[0]
#         else:
#             tmp_new_df['Label'] = [0]*tmp_new_df.shape[0]
#         tmp_new_df['max_ICULOS'] = [tmp_new_df['ICULOS'].values[-1]]*tmp_new_df.shape[0]
#         tmp_new_df['time_bm'] =  tmp_new_df['ICULOS']-tmp_new_df['max_ICULOS']
#         new_df = new_df.append(tmp_new_df)
#     return new_df

In [ ]:
# for d_type in ['train', 'test']:
#     data_path = f'/home/student/Early_Prediction_of_Sepsis/data/{d_type}/'
#     patients = os.listdir(f'data/{d_type}')
#     if d_type=='train':
#         train_patients = sample(patients,int(len(patients)*0.8))
#         val_patients = [x for x in patients if x not in train_patients]
#         train_df = create_patients_df(train_patients,data_path)
#         train_df.to_csv(f'{TRAIN_PATH}.csv',index=False)
#         val_df = create_patients_df(val_patients,data_path)
#         val_df.to_csv(f'{VAL_PATH}.csv',index=False)
#     else:
#         test_df = create_patients_df(patients,data_path)
#         test_df.to_csv(f'{TEST_PATH}.csv',index=False)

In [6]:
frequency_used_attributes = ['BaseExcess',  'FiO2', 'pH', 'PaCO2', 'Glucose','Lactate', 'PTT']
# FREQUENCY_ATTR =['5w_sum_BaseExcess', '5w_sum_FiO2', '5w_sum_pH', '5w_sum_PaCO2', '5w_sum_Glucose', '5w_sum_Lactate', '5w_sum_PTT']
# LAB_ATTR = ['Hct',  'Glucose','Potassium']
CONST_ATTR = ['max_ICULOS','Gender']
OTHER_ATTR = ['HR','MAP','O2Sat', 'Resp','SBP','ICULOS']
ALL_LAB_ATTR = ['BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2',
 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride',
 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate',
 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total',
 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen','Platelets']
COLS = CONST_ATTR+OTHER_ATTR

In [7]:
class data_preperator():
    def __init__(self,columns,window_columns=None, freq_columns=None, seq_len=10,window=5):
        self.all_data_means= pd.read_csv(TRAIN_MEAN_PATH)
        self.seq_len=seq_len
        self.window = window
        self.window_columns = window_columns
        self.freq_columns = freq_columns
        self.columns = columns

    def impute_per_patient(self,df):
        patients = list(set(df.ID.values))
        imputed = pd.DataFrame()
        for patient in patients:
            tmp_df = df[df['ID']==patient][self.columns+['Label']]
            for f in self.columns:
                if tmp_df[f].isnull().all:
                    try:
                        mean_val = self.all_data_means[self.all_data_means['index']==f]['0'].values[0]
                        tmp_df[f]=tmp_df[f].fillna(mean_val)
                    except:
                        print('Exception')
                        print('feature: ', f)
                        print(self.all_data_means[self.all_data_means['index']==f]['0'])
                        print(tmp_df[[f]])
            imp = IterativeImputer(max_iter=50, random_state=0)
            try:
                imp.fit(tmp_df)
                tmp_df= pd.DataFrame(imp.transform(tmp_df), columns = self.columns+['Label'])
                imputed=imputed.append(tmp_df)
            except:
                print(tmp_df.shape)
        return imputed

    def add_rolling_window(self,df):
        df = df.sort_values(by=['ID','ICULOS'], ascending =[True,True])
        rolling = df[['ID']+self.window_columns].groupby('ID').rolling(window=self.window, closed='both').count()
        rolling= rolling.rename(columns={at: f'{self.window}w_sum_{at}' for at in self.window_columns})
        rolling=rolling[list(rolling.columns)[1:]].reset_index().set_index('level_1')
        combined = df.join(rolling,how='left', rsuffix= 'r')
        self.columns +=[f'{self.window}w_sum_{at}' for at in self.window_columns]
        return combined

    def add_frequency(self,df):
        df = df.sort_values(by=['ID','ICULOS'], ascending =[True,True])
        rolling = df[['ID','max_ICULOS']+self.freq_columns].groupby(by=['ID','max_ICULOS'])[self.freq_columns].expanding().count().reset_index()
        for at in self.freq_columns:
            rolling[at] = rolling[at]/rolling['max_ICULOS']
        rolling= rolling.rename(columns={at: f'freq_{at}' for at in self.freq_columns})
        combined = df.join(rolling,how='left',on='ID', rsuffix= 'r')
        self.columns +=[f'freq_{at}' for at in self.freq_columns]
        return combined


    def prepare_data(self,df, rolling=False,freq=True):
        if rolling:
            df = self.add_rolling_window(df)
        if freq:
            df = self.add_frequency(df)
        df = df[df['time_bm']>=-1*(self.seq_len)]
        df = df[self.columns+['Label','ID','time_bm']]
        df = self.impute_per_patient(df)
        return df

In [7]:
train_df = pd.read_csv(f'{TRAIN_PATH}.csv')
all_data_mean = train_df.mean().reset_index().to_csv(TRAIN_MEAN_PATH,index=False)

In [8]:
p = data_preperator(columns=COLS,freq_columns=ALL_LAB_ATTR)

In [45]:
p.all_data_means

,index,0
0,HR,84.486285
1,O2Sat,97.193457
2,Temp,36.968404
3,SBP,123.495401
4,MAP,82.328402
5,DBP,63.834791
6,Resp,18.713982
7,EtCO2,33.240873
8,BaseExcess,-0.807791
9,HCO3,24.100393


In [97]:
# train_df = pd.read_csv(f'{TRAIN_PATH}.csv')
# train_df = p.prepare_data(train_df,rolling=False, freq=True)
# train_df.to_csv(f'{TRAIN_PATH}_LSTM_new.csv')

NameError: name 'p' is not defined

In [3]:
val_df = pd.read_csv(f'{VAL_PATH}.csv')
# val_df = p.prepare_data(val_df)
# # val_df.to_csv(f'{VAL_PATH}_LSTM_new.csv')
# val_df

In [ ]:
test_df = pd.read_csv(f'{TEST_PATH}.csv')
test_df = p.prepare_data(test_df)
test_df.to_csv(f'{TEST_PATH}_LSTM_new.csv')

In [59]:
train_df = pd.read_csv(f'{TRAIN_PATH}.csv')

In [26]:
train_df.sort_values(by=['ID','ICULOS'], ascending =[True,True])

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,ID,Label,max_ICULOS,time_bm
75951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,1,0,0,0,23,-22
75952,61.0,99.0,36.44,124.0,65.0,43.0,17.5,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,2,0,0,0,23,-21
75953,64.0,98.0,NaN,125.0,64.0,41.0,27.0,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,3,0,0,0,23,-20
75954,56.0,100.0,NaN,123.0,65.0,41.0,9.0,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,4,0,0,0,23,-19
75955,66.0,99.0,NaN,120.0,67.0,43.0,23.0,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,5,0,0,0,23,-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154924,76.0,NaN,NaN,85.0,53.0,NaN,17.0,NaN,NaN,NaN,...,0,NaN,NaN,-0.03,50,0,19999,0,54,-4
154925,81.0,99.0,NaN,99.0,51.0,NaN,17.0,NaN,NaN,NaN,...,0,NaN,NaN,-0.03,51,0,19999,0,54,-3
154926,85.0,100.0,NaN,103.0,48.0,NaN,26.0,NaN,NaN,NaN,...,0,NaN,NaN,-0.03,52,0,19999,0,54,-2
154927,86.0,93.0,NaN,87.0,44.0,NaN,22.0,NaN,NaN,NaN,...,0,NaN,NaN,-0.03,53,0,19999,0,54,-1


In [4]:
def add_frequency(df, attr):
    df = df.sort_values(by=['ID','ICULOS'], ascending =[True,True])
    rolling = df[['ID','max_ICULOS','ICULOS']+attr].groupby(by=['ID','max_ICULOS'])[attr].expanding().count().reset_index()
    for at in attr:
        rolling[at] = rolling[at]/rolling['max_ICULOS']
    rolling= rolling.rename(columns={at: f'freq_{at}' for at in attr})
    combined = df.join(rolling,how='left', rsuffix= 'r')
    return combined

In [9]:
attr = ALL_LAB_ATTR
# df = val_df.sort_values(by=['ID', 'ICULOS'], ascending=[True, True])
rolling = df[['ID','max_ICULOS','ICULOS']+attr].groupby(by=['ID','max_ICULOS'])[attr].expanding().count().reset_index()
# for at in attr:
#     rolling[at] = rolling[at] / rolling['max_ICULOS']
# rolling = rolling.rename(columns={at: f'freq_{at}' for at in attr})
# combined = df.join(rolling, how='left', rsuffix='r')
# return combined


In [10]:
rolling

,ID,max_ICULOS,level_2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,...,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets
0,1,26,150546,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,26,150547,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1,26,150548,1.0,1.0,2.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,26,150549,1.0,2.0,2.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,1.0
4,1,26,150550,1.0,2.0,2.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151357,19993,50,21276,3.0,2.0,12.0,5.0,3.0,2.0,0.0,...,2.0,4.0,0.0,0.0,9.0,5.0,2.0,2.0,0.0,2.0
151358,19993,50,21277,3.0,2.0,13.0,5.0,3.0,2.0,0.0,...,2.0,4.0,0.0,0.0,9.0,5.0,2.0,2.0,0.0,2.0
151359,19993,50,21278,3.0,2.0,13.0,5.0,3.0,2.0,0.0,...,3.0,5.0,0.0,0.0,10.0,5.0,2.0,2.0,0.0,2.0
151360,19993,50,21279,3.0,2.0,13.0,5.0,3.0,2.0,0.0,...,3.0,5.0,0.0,0.0,10.0,5.0,2.0,2.0,0.0,2.0


In [12]:
df[['Phosphate']]

,Phosphate
150546,NaN
150547,NaN
150548,NaN
150549,4.2
150550,NaN
...,...
21276,NaN
21277,NaN
21278,3.5
21279,NaN


In [11]:
df = add_frequency(val_df,ALL_LAB_ATTR)

In [14]:
df[['freq_Phosphate','Phosphate','ID','ICULOS']]

,freq_Phosphate,Phosphate,ID,ICULOS
150546,0.036364,NaN,1,2
150547,0.036364,NaN,1,3
150548,0.036364,NaN,1,4
150549,0.036364,4.2,1,5
150550,0.054545,NaN,1,6
...,...,...,...,...
21276,0.000000,NaN,19993,46
21277,0.000000,NaN,19993,47
21278,0.000000,3.5,19993,48
21279,0.000000,NaN,19993,49


In [74]:
df = train_df[train_df.ID.isin([0,1,2,3])].sort_values(by=['ID','ICULOS'], ascending =[True,True])
rolling = df[['ID','max_ICULOS']+ALL_LAB_ATTR].groupby(by=['ID','max_ICULOS'])[ALL_LAB_ATTR].expanding().count().reset_index()


In [78]:
rolling['FiO2'] = rolling['FiO2']/rolling['max_ICULOS']

In [81]:
rolling= rolling.rename(columns={at: f'freq_{at}' for at in ALL_LAB_ATTR})

In [88]:
combined = df.join(rolling,how='left',on='ID', rsuffix= 'r')

In [89]:
combined

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,freq_Phosphate,freq_Potassium,freq_Bilirubin_total,freq_TroponinI,freq_Hct,freq_Hgb,freq_PTT,freq_WBC,freq_Fibrinogen,freq_Platelets
75951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75952,61.0,99.0,36.44,124.0,65.00,43.0,17.5,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75953,64.0,98.0,NaN,125.0,64.00,41.0,27.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75954,56.0,100.0,NaN,123.0,65.00,41.0,9.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75955,66.0,99.0,NaN,120.0,67.00,43.0,23.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539766,58.0,99.0,NaN,152.5,100.00,76.0,14.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539767,68.0,99.0,NaN,128.0,74.33,77.0,17.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539768,71.0,95.0,NaN,146.5,94.50,73.0,17.0,NaN,NaN,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539769,71.0,96.0,37.00,149.0,94.00,73.0,18.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,ID,Label,max_ICULOS,time_bm
75951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,1,0,0,0,23,-22
75952,61.0,99.0,36.44,124.0,65.00,43.0,17.5,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,2,0,0,0,23,-21
75953,64.0,98.0,NaN,125.0,64.00,41.0,27.0,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,3,0,0,0,23,-20
75954,56.0,100.0,NaN,123.0,65.00,41.0,9.0,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,4,0,0,0,23,-19
75955,66.0,99.0,NaN,120.0,67.00,43.0,23.0,NaN,NaN,NaN,...,0,0.0,1.0,-98.60,5,0,0,0,23,-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539766,58.0,99.0,NaN,152.5,100.00,76.0,14.0,NaN,NaN,NaN,...,1,NaN,NaN,-0.02,44,0,3,0,48,-4
539767,68.0,99.0,NaN,128.0,74.33,77.0,17.0,NaN,NaN,NaN,...,1,NaN,NaN,-0.02,45,0,3,0,48,-3
539768,71.0,95.0,NaN,146.5,94.50,73.0,17.0,NaN,NaN,23.0,...,1,NaN,NaN,-0.02,46,0,3,0,48,-2
539769,71.0,96.0,37.00,149.0,94.00,73.0,18.0,NaN,NaN,NaN,...,1,NaN,NaN,-0.02,47,0,3,0,48,-1
